In [ ]:
%pip install opencv-python
%pip install tf_explain

# NN Explainability

In [9]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import shap
import matplotlib.pyplot as plt
from tf_explain.core.grad_cam import GradCAM
from sklearn.calibration import LabelEncoder
import numpy as np
import torch

In [2]:
import os
import sys

original_sys_path = sys.path.copy()
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../general_utils'))
sys.path.append(parent_dir)

from preprocessing import *
from nn_utils import *

sys.path = original_sys_path

[nltk_data] Downloading package punkt_tab to /Users/sara/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
# Load training set
df_train = pd.read_csv('../Model Implementation/data/train_yangswei_85.csv')  
# Load test set
test_df = pd.read_csv('../Model Implementation/data/test_yangswei_85.csv')
dataset_name = 'Yangswei_85'

In [10]:
df_train['text'].apply(preprocess).to_frame()

# Tokenize and pad training data
padded_sequences, tokenizer, vocab_size = tokenize_and_pad(df_train[['text']])

In [11]:
# Preprocess test data
num_classes = 6
test_df['text'].apply(preprocess).to_frame()  

# Tokenize and pad test data using the same tokenizer from training
padded_test_sequences = tokenizer.texts_to_sequences(test_df['text'])
padded_test_sequences = tf.keras.preprocessing.sequence.pad_sequences(padded_test_sequences, maxlen=padded_sequences.shape[1])

# Tokenize and pad test data
padded_sequences, _, _ = tokenize_and_pad(test_df[['text']])

# Set test data
test_data = padded_sequences

#Encode test labels by loading encoder used for training labels
true_labels = test_df['label']
label_classes = np.load('../Model Implementation/data/label_classes.npy', allow_pickle=True)
label_encoder = LabelEncoder()
label_encoder.classes_ = label_classes
test_labels = torch.tensor(label_encoder.transform(true_labels))
test_labels_one_hot_encoded = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)

In [ ]:
# Load the trained model
model_path = ".h5"  # Replace with your .h5 file path
model = load_model(model_path)

In [ ]:
test_loss, test_accuracy = model.evaluate(padded_test_sequences, test_labels_one_hot_encoded, verbose=1)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


predictions_probabilities = model.predict(padded_test_sequences)
predicted_classes = label_encoder.inverse_transform(np.argmax(predictions_probabilities, axis=1))
true_classes = label_encoder.inverse_transform(np.argmax(test_labels_one_hot_encoded, axis=1))

In [ ]:
# Create a DataFrame with the test instances, true labels, and predicted labels
results_df = test_df['text'].copy()
results_df['true_label'] = true_labels.values
results_df['predicted_label'] = predicted_classes

# Filter to find correctly predicted instances
correct_predictions = results_df[results_df['true_label'] == results_df['predicted_label']]
print("Correctly predicted instances:\n", correct_predictions)


In [ ]:
# Initialize GradCAM
explainer = GradCAM()

# Define the target layer to explain (adjust according to your model)
# This should be the name of the last convolutional layer or a layer of interest
target_layer_name = "last_conv_layer_name"  # Replace with actual layer name in your model

# Run the explainer
explanations = explainer.explain(
    validation_data=(input_data, None),  # Input data and optional labels
    model=model,
    layer_name=target_layer_name
)

# Save or display the output
explainer.save(explanations, ".", "grad_cam_output.png")

# Display the visualization
plt.imshow(explanations)
plt.axis("off")
plt.show()

In [ ]:
# Initialize SHAP Explainer
explainer = shap.Explainer(model, input_data)

# Generate SHAP values
shap_values = explainer(input_data)

# Visualize explanations using SHAP
shap.summary_plot(shap_values, input_data)